In [ ]:
# imports
import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
#google_api_key = os.getenv('GOOGLE_API_KEY')


In [3]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
#GOOGLE_MODEL = "gemini-1.5-pro"

In [4]:
system_message = "You are a Python code assistant. Your task is to analyze Python code and generate high-quality, concise comments and docstrings. Follow these guidelines:"
system_message += "Docstrings: Add a docstring for every function, class, and module. Describe the purpose of the function/class, its parameters, and its return value. Keep the description concise but informative, using proper Python docstring conventions (e.g., Google, NumPy, or reStructuredText format)."
system_message += "Inline Comments: Add inline comments only where necessary to clarify complex logic, important steps, or non-obvious behavior. Avoid commenting on obvious operations like x += 1 unless it involves a nuanced concept. Keep comments short, clear, and relevant."
system_message += "General Instructions: Maintain consistency in style and tone. Use technical terminology where appropriate, but ensure clarity for someone with intermediate Python knowledge. Do not over-explain or add redundant comments for self-explanatory code. Follow PEP 257 and PEP 8 standards for style and formatting."


In [5]:
def user_prompt_for(python):
    user_prompt = "Analyze the following Python code and enhance it by adding high-quality, concise docstrings and comments. "
    user_prompt += "Ensure all functions, classes, and modules have appropriate docstrings describing their purpose, parameters, and return values. "
    user_prompt += "Add inline comments only for complex or non-obvious parts of the code. "
    user_prompt += "Follow Python's PEP 257 and PEP 8 standards for documentation and formatting. "
    user_prompt += "Do not modify the code itself; only add annotations.\n\n"
    user_prompt += python
    return user_prompt


In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [9]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [10]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```python\n','').replace('```','')

In [11]:
def stream_google(python):
    # Initialize empty reply string
    reply = ""
    
    # The API for Gemini has a slightly different structure
    gemini = google.generativeai.GenerativeModel(
        model_name=GOOGLE_MODEL,
        system_instruction=system_message
    )
    
    response = gemini.generate_content(
        user_prompt_for(python),
        stream=True
    )
    
    # Process the stream
    for chunk in response:
        # Extract text from the chunk
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```python\n','').replace('```','')

In [12]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_google(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [13]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [16]:
def exibir_codigo_no_output(codigo_para_exibir):
    """
    Recebe uma string de código e a retorna.
    O componente gr.Code se encarregará da formatação e do destaque de sintaxe.
    """
    return codigo_para_exibir

In [17]:
css_new = """
/* --- Estilos Globais --- */
body, .gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background-color: #282c34;
    color: #abb2bf;
    line-height: 1.6;
}

.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background-color: #282c34;
    color: #abb2bf;
    line-height: 1.6;
    border-radius: 10px;
    box-shadow: 0 10px 25px rgba(0,0,0,0.3);
    padding: 25px; /* Padding interno para o conteúdo não colar nas bordas */

    /* --- ALTERAÇÃO PRINCIPAL AQUI PARA MAIS LARGURA --- */
    width: 90%;         /* A interface ocupará 90% da largura da janela */
    max-width: 1800px;  /* Limite máximo de largura para telas muito grandes (ex: 1600px ou 1800px) */
    /* Se quiser que ocupe quase tudo, pode aumentar o max-width ou até remover,
       e usar apenas width: 95%; por exemplo. */
    margin: 30px auto; /* Centraliza o container na página com margens verticais */
}

/* --- Títulos (Markdown) --- */
.prose h2 {
    color: #61afef;
    text-align: center;
    margin-bottom: 30px;
    font-size: 26px;
    font-weight: 600;
    border-bottom: 1px solid #3a3f4b;
    padding-bottom: 15px;
}

/* --- Rótulos dos Componentes (Labels) --- */
.gr-label > .label-text {
    color: #9ea8b7 !important;
    font-size: 0.9em !important;
    font-weight: bold !important;
    margin-bottom: 6px !important;
    text-transform: uppercase;
    letter-spacing: 0.5px;
}

/* --- Caixas de Texto (Textbox) e Dropdown (parte do input) --- */
textarea, input[type="text"], input[type="number"], .gr-dropdown > div[role="combobox"] {
    background-color: #21252b !important;
    color: #dcdde1 !important;
    border: 1px solid #3a3f4b !important;
    border-radius: 6px !important;
    padding: 12px !important;
    font-size: 1em !important;
    box-shadow: inset 0 1px 3px rgba(0,0,0,0.2);
}
textarea::placeholder, input[type="text"]::placeholder {
    color: #5c6370 !important;
    opacity: 1 !important;
}
textarea:focus, input[type="text"]:focus, input[type="number"]:focus, .gr-dropdown > div[role="combobox"]:focus-within {
    border-color: #61afef !important;
    box-shadow: 0 0 0 3px rgba(97, 175, 239, 0.2) !important;
}

/* --- Componente de Código (gr.Code) --- */
.gr-code {
    border-radius: 6px !important;
    border: 1px solid #3a3f4b !important;
    box-shadow: 0 4px 8px rgba(0,0,0,0.2);
}
.gr-code pre {
    background-color: #1e2227 !important;
    padding: 18px !important;
    border-radius: 6px !important;
}
.gr-code pre code {
    color: #abb2bf !important;
    font-family: 'Fira Code', 'Consolas', 'Monaco', monospace !important;
    font-size: 0.95em !important;
    line-height: 1.7;
}

/* --- Botões (Button) --- */
.gr-button {
    background-image: linear-gradient(to right, #61afef, #5691c8) !important;
    color: #ffffff !important;
    border: none !important;
    border-radius: 6px !important;
    padding: 12px 24px !important;
    font-size: 1.05em !important;
    font-weight: bold !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 4px 8px rgba(0,0,0,0.15);
    letter-spacing: 0.5px;
    /* text-transform: capitalize; REMOVIDO para melhor visualização dos ícones unicode */
}
.gr-button:hover {
    background-image: linear-gradient(to right, #5298d8, #4a81b3) !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 6px 12px rgba(0,0,0,0.2) !important;
}
.gr-button:active {
    transform: translateY(0px) !important;
    box-shadow: 0 2px 4px rgba(0,0,0,0.15) !important;
}

/* --- Lista do Dropdown --- */
.gr-dropdown ul {
    background-color: #2c313a !important;
    border: 1px solid #4a505c !important;
    border-radius: 6px;
    box-shadow: 0 4px 10px rgba(0,0,0,0.2);
}
.gr-dropdown ul li {
    color: #abb2bf !important;
    padding: 12px !important;
    font-size: 0.95em;
}
.gr-dropdown ul li:hover, .gr-dropdown ul li.selected {
    background-color: #3a3f4b !important;
    color: #ffffff !important;
}

/* --- Linhas (Row) --- */
.gr-row {
    margin-bottom: 20px;
    align-items: center;
}
"""

In [ ]:
# Interface Gradio
with gr.Blocks(css=css_new) as ui:
    gr.Markdown("## Code commenter") # Mantendo o título original do seu snippet
    with gr.Row():
        python = gr.Textbox(label="Código Python:", value="", lines=10)
        commented_python = gr.Textbox(label="Código Comentado:", lines=10, placeholder="O código Python após ser comentado pelo modelo aparecerá aqui.")
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Selecionar modelo", value="GPT")
    with gr.Row():
        comment = gr.Button("Comentar código") # Botão para gerar o código comentado
    with gr.Row():
        # Botão para MOSTRAR o código comentado no campo de visualização abaixo
        # A etiqueta foi alterada para "Visualizar Código Comentado" para maior clareza
        python_display_button = gr.Button("Visualizar Código Comentado")
    with gr.Row():
        # Campo para exibir o CÓDIGO COMENTADO com destaque de sintaxe
        # Usando gr.Code para formatação e destaque de sintaxe do Python
        python_out = gr.Code(label="Visualização do Código Comentado:", language="python", interactive=False)

    # Ação do botão "Comentar código" (permanece a mesma)
    # Esta função preenche o campo 'commented_python'
    comment.click(
        optimize,
        inputs=[python, model],
        outputs=[commented_python]
    )

    # Ação do botão "Visualizar Código Comentado"
    # Pega o conteúdo de 'commented_python' e o exibe em 'python_out'
    # A função 'exibir_codigo_no_output' apenas repassa o valor.
    python_display_button.click(
        exibir_codigo_no_output, # Função que simplesmente retorna o input
        inputs=[commented_python],
        outputs=[python_out]
    )

# Lançar a interface
if __name__ == "__main__":
    ui.launch(inbrowser=True)